# working on a pretrained model

## pretrained models as feature extractors

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
model = ResNet50(weights='imagenet',include_top=False)
img_path = 'd:\\tests\\cat0.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
preds = model.predict(x)

FileNotFoundError: ignored

In [ ]:
preds.shape

NameError: ignored

###### https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b
### Transfer learning for CIFAR using resnet50 of 10 classes

In [ ]:
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
def preprocess_data(X, Y):
    """trains a convolutional neural network to classify the dataset"""
    X_p = K.applications.resnet50.preprocess_input(X)
    Y_p = K.utils.to_categorical(Y, 10)
    return X_p, Y_p

In [ ]:
(trainX, trainy), (testX, testy) = K.datasets.cifar10.load_data()
trainX, trainy = preprocess_data(trainX, trainy)
testX, testy = preprocess_data(testX, testy)

In [ ]:
#optionally reduce the data size
import numpy as np
Nrequired=20000
r=np.random.permutation(trainX.shape[0])
print(trainX.shape)
print(trainy.shape)
trainX=trainX[r[0:Nrequired],:,:,:]
trainy=trainy[r[0:Nrequired]]
r=np.random.permutation(testX.shape[0])
testX=testX[r[0:Nrequired],:,:,:]
testy=testy[r[0:Nrequired]]
print(trainX.shape)
print(trainy.shape)

(50000, 32, 32, 3)
(50000, 10)
(20000, 32, 32, 3)
(20000, 10)


In [ ]:
inputs = K.Input(shape=(224, 224, 3))
#Loading the ResNet50 model with pre-trained ImageNet weights
resnet = K.applications.ResNet50(weights='imagenet',include_top=False,input_tensor=inputs)

In [ ]:
for layer in resnet.layers[:170]:
    layer.trainable = False

In [ ]:
model = K.models.Sequential()
model.add(K.layers.Lambda(lambda x: tf.image.resize(x,(224, 224))))
model.add(resnet)

model.add(K.layers.Flatten())

model.add(K.layers.Dense(64, activation='relu'))
model.add(K.layers.Dropout(0.3))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='binary_crossentropy',
     optimizer=K.optimizers.RMSprop(lr=2e-5), metrics=['accuracy'])
model.fit(trainX, trainy, batch_size=64, epochs=30, verbose=1,
     validation_data=(testX, testy),
     shuffle=True)
model.summary()
# model.save("cifar10.h5")

Epoch 1/30


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


32/32 [==============================] - 25s 396ms/step - loss: 0.7666 - accuracy: 0.4210 - val_loss: 0.8253 - val_accuracy: 0.6655
Epoch 2/30
32/32 [==============================] - 11s 356ms/step - loss: 0.6519 - accuracy: 0.7565 - val_loss: 0.7391 - val_accuracy: 0.7485
Epoch 3/30
32/32 [==============================] - 11s 365ms/step - loss: 0.6094 - accuracy: 0.8525 - val_loss: 0.6484 - val_accuracy: 0.7805
Epoch 4/30
32/32 [==============================] - 12s 372ms/step - loss: 0.5763 - accuracy: 0.9165 - val_loss: 0.6517 - val_accuracy: 0.7935
Epoch 5/30
32/32 [==============================] - 12s 386ms/step - loss: 0.5527 - accuracy: 0.9535 - val_loss: 0.6155 - val_accuracy: 0.8125
Epoch 6/30
32/32 [==============================] - 13s 402ms/step - loss: 0.5379 - accuracy: 0.9695 - val_loss: 0.5947 - val_accuracy: 0.8160
Epoch 7/30
32/32 [==============================] - 13s 415ms/step - loss: 0.5262 - accuracy: 0.9795 - val_loss: 0.5721 - val_accuracy: 0.8250
Epoch 8/30

In [ ]:
image_pred=model.predict(testX)
np.argmax(image_pred,axis=1)

In [ ]:
model = K.models.Sequential()
model.add(K.layers.Lambda(lambda x: tf.image.resize(x,(224, 224))))
model.add(resnet)

model.add(K.layers.GlobalAveragePooling2D())

model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(64, activation='relu'))
model.add(K.layers.Dropout(0.3))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='binary_crossentropy',
     optimizer=K.optimizers.RMSprop(lr=2e-5), metrics=['accuracy'])
checkpointer = K.callbacks.ModelCheckpoint(filepath='cifar10.h5',
     monitor="val_accuracy", verbose=1, save_best_only=True)
model.fit(trainX, trainy, batch_size=128, epochs=20, verbose=1,
     callbacks=[checkpointer], validation_data=(testX, testy),
     shuffle=True)
model.summary()
# model.save("cifar10.h5")

Epoch 1/20
157/157 [==============================] - ETA: 0s - loss: 0.6811 - accuracy: 0.6119
Epoch 1: val_accuracy improved from -inf to 0.76880, saving model to cifar10.h5
157/157 [==============================] - 104s 629ms/step - loss: 0.6811 - accuracy: 0.6119 - val_loss: 0.5886 - val_accuracy: 0.7688
Epoch 2/20
157/157 [==============================] - ETA: 0s - loss: 0.6265 - accuracy: 0.7363
Epoch 2: val_accuracy improved from 0.76880 to 0.81250, saving model to cifar10.h5
157/157 [==============================] - 106s 676ms/step - loss: 0.6265 - accuracy: 0.7363 - val_loss: 0.5524 - val_accuracy: 0.8125
Epoch 3/20
157/157 [==============================] - ETA: 0s - loss: 0.5928 - accuracy: 0.7893
Epoch 3: val_accuracy improved from 0.81250 to 0.83150, saving model to cifar10.h5
157/157 [==============================] - 96s 612ms/step - loss: 0.5928 - accuracy: 0.7893 - val_loss: 0.5269 - val_accuracy: 0.8315
Epoch 4/20
157/157 [==============================] - ETA: 0s 

In [ ]:
image_pred=model.predict(testX)
np.argmax(image_pred,axis=1)